# Analisis Model Keuangan Perusahaan

Notebook ini bertujuan untuk melakukan analisis keuangan perusahaan menggunakan beberapa model skor seperti Beneish M-Score, Altman Z-Score, dan rasio keuangan umum untuk penilaian risiko kredit.

## 1. Setup dan Import Library

In [1]:
import json
import sys
import os

# Tambahkan path ke direktori FinancialScoreModels jika notebook ini tidak berada di root yang sama
# module_path = os.path.abspath(os.path.join('..')) # Jika notebook di dalam subfolder
# if module_path not in sys.path:
#     sys.path.append(module_path)

from FinancialScoreModels.utils import load_financial_data
from FinancialScoreModels.beneish_m_score import calculate_beneish_m_score
from FinancialScoreModels.altman_z_score import calculate_altman_z_score
from FinancialScoreModels.financial_ratios import calculate_common_financial_ratios

## 2. Konfigurasi Path File Data Keuangan

Atur path ke file JSON yang berisi hasil ekstraksi data keuangan untuk periode `t` (tahun berjalan) dan `t-1` (tahun sebelumnya).

In [2]:
FILE_PATH_T = "Output/Sarana/hasil_ekstraksi_semua_dokumen.json"
FILE_PATH_T_MINUS_1 = "Output/Sarana/hasil_ekstraksi_semua_dokumen_t_minus_1.json"

## 3. Memuat Data Keuangan

In [3]:
print(f"Memuat data keuangan dari: {FILE_PATH_T}")
data_t = load_financial_data(FILE_PATH_T)

print(f"Memuat data keuangan dari: {FILE_PATH_T_MINUS_1}")
data_t_minus_1 = load_financial_data(FILE_PATH_T_MINUS_1)

if data_t:
    print("\nData Keuangan Periode t berhasil dimuat.")
    # print(json.dumps(data_t, indent=2, ensure_ascii=False))
else:
    print("\nGagal memuat data keuangan periode t.")

if data_t_minus_1:
    print("\nData Keuangan Periode t-1 berhasil dimuat.")
    # print(json.dumps(data_t_minus_1, indent=2, ensure_ascii=False))
else:
    print("\nGagal memuat data keuangan periode t-1.")

Memuat data keuangan dari: Output/Sarana/hasil_ekstraksi_semua_dokumen.json
Memuat data keuangan dari: Output/Sarana/hasil_ekstraksi_semua_dokumen_t_minus_1.json

Data Keuangan Periode t berhasil dimuat.

Data Keuangan Periode t-1 berhasil dimuat.


## 4. Perhitungan Beneish M-Score

Beneish M-Score digunakan untuk mendeteksi potensi manipulasi laba.

In [4]:
if data_t and data_t_minus_1:
    print("\n--- Menghitung Beneish M-Score ---")
    m_score, m_ratios = calculate_beneish_m_score(data_t, data_t_minus_1)
    
    if m_score is not None:
        print(f"Beneish M-Score: {m_score:.4f}")
        print("Rincian Rasio Beneish:")
        for ratio_name, ratio_value in m_ratios.items():
            if ratio_name != 'error':
                print(f"  {ratio_name}: {ratio_value:.4f}")
        
        # Interpretasi Umum
        if m_score > -1.78: # Batas umum yang sering dikutip
            print("Interpretasi: Kemungkinan besar perusahaan adalah manipulator laba.")
        elif m_score > -2.22:
            print("Interpretasi: Zona abu-abu, perlu investigasi lebih lanjut.")
        else:
            print("Interpretasi: Kemungkinan kecil perusahaan adalah manipulator laba.")
    else:
        print("Gagal menghitung Beneish M-Score.")
        if m_ratios and 'error' in m_ratios:
            print(f"  Alasan: {m_ratios['error']}")
else:
    print("\nData tidak cukup untuk menghitung Beneish M-Score (data t atau t-1 tidak berhasil dimuat).")


--- Menghitung Beneish M-Score ---
Beneish M-Score: -1.4890
Rincian Rasio Beneish:
  DSRI: 0.9654
  GMI: 1.0841
  AQI: 0.9858
  SGI: 1.1395
  DEPI: 1.0000
  SGAI: 0.9654
  LVGI: 1.0134
  TATA: 0.1834
Interpretasi: Kemungkinan besar perusahaan adalah manipulator laba.


## 5. Perhitungan Altman Z-Score

Altman Z-Score digunakan untuk memprediksi probabilitas kebangkrutan perusahaan.

In [5]:
if data_t:
    print("\n--- Menghitung Altman Z-Score ---")
    # Asumsikan perusahaan adalah manufaktur publik untuk contoh ini
    # Ganti model_type jika perlu: "private_manufacturing", "non_manufacturing_or_emerging_markets"
    z_model_type = "public_manufacturing" 
    z_score, z_ratios = calculate_altman_z_score(data_t, model_type=z_model_type)
    
    if z_score is not None:
        print(f"Altman Z-Score (Model: {z_ratios.get('model_type', z_model_type)}): {z_score:.4f}")
        print("Rincian Rasio Altman:")
        for ratio_name, ratio_value in z_ratios.items():
            if ratio_name not in ['error', 'model_type', 'interpretation_zones', 'X4_note']:
                print(f"  {ratio_name}: {ratio_value:.4f}")
        if "X4_note" in z_ratios:
            print(f"  Catatan untuk X4: {z_ratios['X4_note']}")
            
        zones = z_ratios.get("interpretation_zones", {})
        print("Interpretasi Zona:")
        if z_model_type == "public_manufacturing":
            if z_score > 2.99: print("  Perusahaan berada di 'Safe Zone'.")
            elif z_score > 1.81: print("  Perusahaan berada di 'Grey Zone'.")
            else: print("  Perusahaan berada di 'Distress Zone'.")
        elif z_model_type == "private_manufacturing":
            if z_score > 2.90: print("  Perusahaan berada di 'Safe Zone'.")
            elif z_score > 1.23: print("  Perusahaan berada di 'Grey Zone'.")
            else: print("  Perusahaan berada di 'Distress Zone'.")
        elif z_model_type == "non_manufacturing_or_emerging_markets":
            if z_score > 2.60: print("  Perusahaan berada di 'Safe Zone'.")
            elif z_score > 1.10: print("  Perusahaan berada di 'Grey Zone'.")
            else: print("  Perusahaan berada di 'Distress Zone'.")
        print(f"  Detail Zona (dari model): {zones}")
    else:
        print("Gagal menghitung Altman Z-Score.")
        if z_ratios and 'error' in z_ratios:
            print(f"  Alasan: {z_ratios['error']}")
else:
    print("\nData tidak cukup untuk menghitung Altman Z-Score (data t tidak berhasil dimuat).")


--- Menghitung Altman Z-Score ---
Altman Z-Score (Model: Public Manufacturing (1968)): 5.8919
Rincian Rasio Altman:
  X1 (Working Capital / Total Assets): 0.0489
  X2 (Retained Earnings / Total Assets): 0.6435
  X3 (EBIT / Total Assets): 0.2246
  X4 (Market Value of Equity / Total Liabilities): 5.2007
  X5 (Sales / Total Assets): 1.0717
  Catatan untuk X4: Using Book Value of Equity as proxy for Market Value
Interpretasi Zona:
  Perusahaan berada di 'Safe Zone'.
  Detail Zona (dari model): {'Safe Zone': '> 2.99', 'Grey Zone': '1.81 - 2.99', 'Distress Zone': '< 1.81'}


## 6. Perhitungan Rasio Keuangan Umum (untuk Analisis Risiko Kredit)

Rasio-rasio ini memberikan gambaran umum tentang kesehatan keuangan, likuiditas, solvabilitas, dan profitabilitas perusahaan.

In [6]:
if data_t:
    print("\n--- Menghitung Rasio Keuangan Umum ---")
    common_ratios = calculate_common_financial_ratios(data_t)
    
    if "error" in common_ratios:
        print(f"Gagal menghitung rasio keuangan umum: {common_ratios['error']}")
    else:
        print("Rasio Keuangan Umum:")
        for ratio_name, ratio_value in common_ratios.items():
            if not ratio_name.endswith("_note") and not ratio_name.endswith("_error"):
                # Check if ratio_value is a number before formatting
                if isinstance(ratio_value, (int, float)):
                    print(f"  {ratio_name}: {ratio_value:.4f}", end="")
                else:
                    print(f"  {ratio_name}: {ratio_value}", end="")
                
                if f"{ratio_name}_note" in common_ratios:
                    print(f" ({common_ratios[f'{ratio_name}_note']})")
                else:
                    print()
            elif ratio_name.endswith("_error"):
                 print(f"  Error untuk {ratio_name.replace('_error', '')}: {ratio_value}")
        
        # Contoh Sederhana untuk Credit Risk Score Indication
        # Ini adalah contoh yang sangat dasar dan perlu disesuaikan dengan model risiko kredit yang lebih canggih.
        print("\nContoh Indikasi Skor Risiko Kredit (Sangat Sederhana):")
        score_points = 0
        reasons = []

        # Debt-to-Equity Ratio
        dte = common_ratios.get("Debt-to-Equity Ratio")
        if isinstance(dte, float):
            if dte < 0.5: score_points += 2; reasons.append(f"D/E Rendah ({dte:.2f})")
            elif dte < 1.0: score_points += 1; reasons.append(f"D/E Sedang ({dte:.2f})")
            else: score_points -=1; reasons.append(f"D/E Tinggi ({dte:.2f})")
        
        # Current Ratio
        cr = common_ratios.get("Current Ratio")
        if isinstance(cr, float):
            if cr > 2.0: score_points += 2; reasons.append(f"Current Ratio Baik ({cr:.2f})")
            elif cr > 1.0: score_points += 1; reasons.append(f"Current Ratio Cukup ({cr:.2f})")
            else: score_points -=1; reasons.append(f"Current Ratio Rendah ({cr:.2f})")

        # Interest Coverage Ratio
        icr = common_ratios.get("Interest Coverage Ratio")
        if isinstance(icr, float):
            if icr == float('inf'): score_points +=2; reasons.append(f"ICR Sangat Baik (Tak Terhingga)") # No interest, good EBIT
            elif icr > 5.0: score_points += 2; reasons.append(f"ICR Baik ({icr:.2f})")
            elif icr > 2.0: score_points += 1; reasons.append(f"ICR Cukup ({icr:.2f})")
            else: score_points -=1; reasons.append(f"ICR Rendah ({icr:.2f})")
            
        # Net Profit Margin
        npm = common_ratios.get("Net Profit Margin")
        if isinstance(npm, float):
            if npm > 0.1: score_points +=2; reasons.append(f"NPM Baik ({npm:.2%})")
            elif npm > 0.05: score_points +=1; reasons.append(f"NPM Cukup ({npm:.2%})")
            elif npm > 0: score_points +=0; reasons.append(f"NPM Positif Rendah ({npm:.2%})")
            else: score_points -=1; reasons.append(f"NPM Negatif ({npm:.2%})")
            
        print(f"Total Poin Skor Kredit (Contoh): {score_points}")
        print(f"Alasan Poin: {', '.join(reasons)}")
        
        if score_points >= 5:
            print("Indikasi Risiko Kredit: Rendah")
        elif score_points >= 2:
            print("Indikasi Risiko Kredit: Sedang")
        else:
            print("Indikasi Risiko Kredit: Tinggi")
else:
    print("\nData tidak cukup untuk menghitung Rasio Keuangan Umum (data t tidak berhasil dimuat).")


--- Menghitung Rasio Keuangan Umum ---
Rasio Keuangan Umum:
  Debt-to-Equity Ratio: 0.1923
  Current Ratio: 1.3453
  Interest Coverage Ratio: 41.2473
  Net Profit Margin: 0.2001
  Gross Profit Margin: 0.0971
  Debt Ratio: 0.1613

Contoh Indikasi Skor Risiko Kredit (Sangat Sederhana):
Total Poin Skor Kredit (Contoh): 7
Alasan Poin: D/E Rendah (0.19), Current Ratio Cukup (1.35), ICR Baik (41.25), NPM Baik (20.01%)
Indikasi Risiko Kredit: Rendah


## 7. Kesimpulan Analisis

Berdasarkan hasil perhitungan di atas:
*   **Beneish M-Score**: Memberikan indikasi apakah ada potensi manipulasi laba.
*   **Altman Z-Score**: Memberikan prediksi mengenai risiko kebangkrutan.
*   **Rasio Keuangan Umum**: Memberikan gambaran kesehatan keuangan secara umum dan dapat digunakan sebagai dasar penilaian risiko kredit.

Analisis ini sebaiknya digunakan sebagai salah satu alat bantu dalam pengambilan keputusan dan dikombinasikan dengan analisis kualitatif serta informasi pasar lainnya.